# National Park Trails ETL
This notebook is a helper file for etl processing into the database.

## Part I. View the Data

In [2]:
# import packages

import pandas as pd
import cassandra

In [3]:
df = pd.read_csv("../data/alltrails-nationalpark.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3313 entries, 0 to 3312
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   trail_id           3313 non-null   int64  
 1   name               3313 non-null   object 
 2   area_name          3313 non-null   object 
 3   city_name          3313 non-null   object 
 4   state_name         3313 non-null   object 
 5   country_name       3313 non-null   object 
 6   _geoloc            3313 non-null   object 
 7   popularity         3313 non-null   float64
 8   length             3313 non-null   float64
 9   elevation_gain     3313 non-null   float64
 10  difficulty_rating  3313 non-null   int64  
 11  route_type         3313 non-null   object 
 12  visitor_usage      3060 non-null   float64
 13  avg_rating         3313 non-null   float64
 14  num_reviews        3313 non-null   int64  
 15  features           3313 non-null   object 
 16  activities         3313 

In [4]:
df.head(2)

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18852, 'lng': -149.63156}",24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,"['dogs-no', 'forest', 'river', 'views', 'water...","['birding', 'camping', 'hiking', 'nature-trips...",i
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73049, 'lng': -148.91968}",18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,"['dogs-no', 'forest', 'views', 'wild-flowers',...","['birding', 'camping', 'hiking', 'nature-trips...",i


## Set up Apache Cassandra

* Creating a cluster
* Creating and setting up the keyspace


In [5]:
from cassandra.cluster import Cluster

# create the cluster to connect to the Cassandra instance
cluster = Cluster(['127.0.0.1'])

# create the connection
session = cluster.connect()

In [1]:
# create the keyspace
create_keyspace = """
    CREATE KEYSPACE IF NOT EXISTS np_trails
    WITH replication = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """

try:
    session.execute(create_keyspace)
except Exception as er:
    print(er)

In [6]:
# set the keyspace
session.set_keyspace('np_trails')

## Setup Queries
Writing a list of questions that would be needed to decipher how to create the Cassandra tables.

1. What is the most popular national park trail?
2. Which 6 national parks have a trail that is considered to have the highest elevation gain amongst other parks?
3. List medium hikes (level 2 or 3) located in Arizona. 

In [75]:
# Drop Table
def drop_table(table_name):
    drop_table_query = "DROP TABLE {}".format(table_name)
    try:
        session.execute(drop_table_query)
    except Exception as er:
        print(er)

#### Query 1: What is the most popular national park?

In [77]:
drop_table("trail_popularity")

create_table_query = """
    CREATE TABLE IF NOT EXISTS trail_popularity 
    (name TEXT, area_name TEXT, popularity FLOAT, 
    PRIMARY KEY(name, popularity)) WITH CLUSTERING ORDER BY (popularity DESC)
    """

try:
    session.execute(create_table_query)
except Exception as er:
    print(er)

In [79]:
# load data into the popularity tables
load_query = """
    INSERT INTO trail_popularity (name, area_name, popularity) 
    VALUES (%s, %s, %s)
    """

for idx, row in df.iterrows():
    session.execute(load_query, (row['name'], row['area_name'], row.popularity))

In [80]:
# query the data
select_query = "SELECT * FROM trail_popularity LIMIT 2"
try:
    data = session.execute(select_query)
except Exception as er:
    print(er)
most_popular_trail = ([list(data.one())])  
pd.DataFrame(most_popular_trail,columns = ['trail_name', 'park_name', 'popularity'])

,trail_name,park_name,popularity
0,Cottonwood-Marble Canyon Loop,9.985,Death Valley National Park


#### Query 2: Which 6 national parks have a trail that is considered to have the highest elevation gain amongst other parks?

In [154]:
drop_table("top_elevation_gain_parks")

In [156]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS top_elevation_gain_parks 
    (area_name TEXT, elevation_gain FLOAT, 
    PRIMARY KEY(area_name, elevation_gain));
    """

try:
    session.execute(create_table_query)
except Exception as er:
    print(er)

In [162]:
# load data into the top_elevation_gain_parks table
load_query = """
    INSERT INTO top_elevation_gain_parks (area_name, elevation_gain) 
    VALUES (%s, %s)
    """

for idx, row in df.iterrows():
    session.execute(load_query, (row['area_name'], row['elevation_gain']))

In [166]:
# query the data
select_query = """
    SELECT area_name, MAX(elevation_gain) AS max
    FROM top_elevation_gain_parks 
    GROUP BY area_name
    """
try:
    result = session.execute(select_query)
except Exception as er:
    print(er)    
    
data = list(map(lambda row: list(row), result))
df = pd.DataFrame(data, columns = ['park', 'max_elevation_gain'])
df.sort_values('max_elevation_gain', ascending = False).iloc[0:6]

,park,max_elevation_gain
2,Yosemite National Park,14029.944336
27,Kings Canyon National Park,10944.758789
50,Olympic National Park,8369.807617
47,Shenandoah National Park,8115.909668
31,Mount Rainier National Park,7752.892578
58,Glacier National Park,5697.931152


#### Query 3: List medium hikes (level 2 or 3) located in Arizona. 

In [8]:
drop_table("medium_trails_in_arizona")

NameError: name 'drop_table' is not defined

In [11]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS medium_trails_in_arizona
    (name TEXT, area_name TEXT, state_name TEXT, difficulty_rating FLOAT, 
    PRIMARY KEY (name));
    """

try:
    session.execute(create_table_query)
except Exception as er:
    print(er)